In [1]:
import pandas as pd
import glob
from tqdm import tqdm
from dask.dataframe import from_pandas
import dask.dataframe as dd

In [3]:
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame

In [4]:
Training_data_files=glob.glob('TrainingData/*.csv')
data_frame_x=pd.DataFrame([])
data_frame_x_time=pd.DataFrame([])
data_frame_y=pd.DataFrame([])
data_frame_y_time=pd.DataFrame([])
for file in tqdm(Training_data_files):
    file_name=file.split('/')[-1]
    temp_frame=pd.read_csv(file,header=None)
    if 'x.csv' in file_name:
        data_frame_x=pd.concat([data_frame_x,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    elif 'x_time.csv' in file_name:
        data_frame_x_time=pd.concat([data_frame_x_time,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    elif 'y.csv' in file_name:
        data_frame_y=pd.concat([data_frame_y,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    else:
        data_frame_y_time=pd.concat([data_frame_y_time,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
data_frame_x = from_pandas(data_frame_x, npartitions=3)
data_frame_x_time=from_pandas(data_frame_x_time.rename(columns={0:'time_stamp'}) , npartitions=3)
data_frame_y=from_pandas(data_frame_y.rename(columns={0:'labels'}), npartitions=3)
data_frame_y_time=from_pandas(data_frame_y_time.rename(columns={0:'time_stamp'}), npartitions=3)


total_x=dd.merge(data_frame_x, data_frame_x_time,on=['session','subject'],npartitions=3) 
total_y=dd.merge(data_frame_y, data_frame_y_time,on=['session','subject'],npartitions=3) 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:08<00:00, 13.19it/s]


In [15]:
data_frame_x.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 5 graph layers